In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="qwen/qwen3-32b")

e:\projects\agentic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### To set a command from one agent to another agent

In [3]:
from langgraph.types import Command

### We have a inbuilt ReactAgent from langgraph 

In [4]:
from langgraph.prebuilt import create_react_agent

### Using this command itself we can switch to multiple agents
    - goto : Where to switch to next agent
    - update : What to update in the state


In [ ]:
def add_number(state):
    result =state["num1"] + state["num2"]
    return Command(goto="multiply", update={"sum":result})

In [6]:
state = {"num1":10, "num2":20}

In [7]:
add_number(state)

Command(update={'sum': 30}, goto='mutiply')

### Creating one dummy MultiAgent
    -> This is for Network Agent or Collaborative Agent

In [9]:
from langchain_core.tools import tool
@tool
def transfer_to_mutiplication_expert():
    """Ask multiplication agnet for help"""
    return 

In [10]:
@tool
def transfer_to_addition_expert():
    """Ask addition agnet for help"""
    return 

In [13]:
llm_with_tools = llm.bind_tools([transfer_to_addition_expert, transfer_to_mutiplication_expert])

In [14]:
response = llm_with_tools.invoke("Hi")

In [15]:
response.content

'Hello! How can I assist you today?'

In [17]:
response.tool_calls

[]

In [18]:
response = llm_with_tools.invoke("what is 2*6")

In [19]:
response.content

''

In [21]:
response.tool_calls

[{'name': 'transfer_to_mutiplication_expert',
  'args': {},
  'id': '3qhndj5s0',
  'type': 'tool_call'}]

In [29]:
from typing_extensions import Literal
from langgraph.graph import MessagesState, StateGraph, START, END

In [ ]:
def addtional_expert(state:MessagesState) -> Command[Literal["mutiplication_expert","__end__"]]:
    

In [33]:
def mutiplication_expert(state:MessagesState) -> Command[Literal["addtional_expert","__end__"]]:
    pass

In [34]:
graph = StateGraph(MessagesState)

In [35]:
graph.add_node("addition_agent",addtional_expert)

In [36]:
graph.add_node("mutiplication_agent",mutiplication_expert)

In [38]:
graph.add_edge(START, "addition_agent")


In [39]:
app = graph.compile()

ValueError: Found edge ending at unknown node `mutiplication_expert`